In [85]:
%%configure -f
{"driverMemory": "4G", "driverCores": 2, "executorMemory": "12G", "executorCores": 6, "numExecutors": 3}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
9,None,pyspark,idle,,,None,✔


In [86]:
from typing import List

from pyspark import SparkFiles
from subprocess import call
import sys


def install_deps(deps: List[str]) -> None:
    call([sys.executable, '-m', 'pip', 'install', '-q', '-t', SparkFiles.getRootDirectory(), *deps])


install_deps(['numpy', 'matplotlib', 'pandas', 'scipy', 'seaborn', 'statsmodels', 'pyarrow'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
from typing import Optional
import json
from pyspark.sql import SparkSession, DataFrame

spark: SparkSession

def load_distance_calculation_df(aggregation_pipeline: Optional[List] = None) -> DataFrame:
    if aggregation_pipeline is None:
        aggregation_pipeline = []
    
    return (
        spark
        .read
        .format("mongodb")
        .option("partitioner", "com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner")
        .option("partitioner.options.partition.field", "_id")
        .option("partitioner.options.partition.size", "64")
        .option("partitioner.options.samples.per.partition", "2")
        .option("database", "enhancer3d")
        .option("collection", "distance_calculation")
        .option("aggregation.pipeline", json.dumps(aggregation_pipeline))
        .load()
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
links_gm12878_df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/work/data/links/GM12878_EP_hg38_liftovered.parquet")
    .alias("links")
)

links_hffc6_df = (
    spark
    .read
    .format("parquet")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/work/data/links/HFFC6_EP_hg38_liftovered.parquet")
    .alias("links")
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [59]:
%%pretty
links_gm12878_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

enh_id,gene_id,pval,qval
chr1:843600-844990,ENSG00000197049,1.180354,1.0
chr1:966620-966700,ENSG00000197049,0.642743,2.0
chr1:843600-844990,ENSG00000188976,1.016813,3.0
chr1:966620-966700,ENSG00000188976,2.58213,4.0
chr1:1013580-1013980,ENSG00000188976,2.073247,5.0


In [89]:
from pyspark.sql import functions as F, types as T
import numpy as np
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

@F.udf(T.ArrayType(T.DoubleType()))
def diff(A, B):
    return np.abs(np.array(A) - np.array(B)).tolist()

@F.udf(T.DoubleType())
def var(A):
    return float(np.var(A))

@F.udf(T.DoubleType())
def avg(A):
    return float(np.mean(A))

@F.udf(T.DoubleType())
def mannwhiteneyu(ref, mod):
    result = stats.mannwhitneyu(np.array(ref), np.array(mod), alternative='two-sided')
    return float(result.pvalue)

@F.udf(T.DoubleType())
def bonferroni_correction(pvalues, alpha=0.05):
    reject, pvals_corrected, _, _ = multipletests(pvalues, alpha=alpha, method='bonferroni')
    return float(np.mean(pvals_corrected))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [100]:
gm12878_neanderthal_df_ref = (
    load_distance_calculation_df([
        {
            '$match': {
                '_id.project_id': '8k_models_project_GM12878',
                '_id.ensemble_id': {'$regex': '^models3D_GM12878_Nean_models3D_GM12878_Nean_results'},
            }
        }
    ])
    .where(
        (F.col('gene_type') == 'protein_coding')
        # & (F.col('enh_tSS_distance') < 20_000)
    )
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        # 'dist',
        'avg_dist',
        'enh_tSS_distance'
    )
    # gene_id ENH00001.XXX -> ENH00001
    .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
    .alias("gm12878")
    .cache()
)

# gm12878_neanderthal_df_ref2 = (
#     load_distance_calculation_df([
#         {
#             '$match': {
#                 '_id.project_id': '8k_models_project_GM12878',
#                 '_id.ensemble_id': {'$regex': '^models3D_GM12878_Nean_models3D_GM12878_Nean_ref2_results'},
#             }
#         }
#     ])
#     .where(
#         (F.col('gene_type') == 'protein_coding')
#         # & (F.col('enh_tSS_distance') < 20_000)
#     )
#     .select(
#         F.col('_id.region_id').alias('region_id'),
#         F.col('_id.gene_id').alias('gene_id'),
#         F.col('_id.enh_id').alias('enh_id'),
#         'dist',
#         'avg_dist',
#         'enh_tSS_distance'
#     )
#     # gene_id ENH00001.XXX -> ENH00001
#     .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
#     .alias("gm12878_ref2")
#     .cache()
# )

hffc6_neanderthal_df_ref = (
    load_distance_calculation_df([
        {
            '$match': {
                '_id.project_id': '8k_models_project_HFFC6',
                '_id.ensemble_id': {'$regex': '^models3D_HFFC6_Nean_models3D_HFFC6_Nean_results'},
            }
        }
    ])
    .where(
        (F.col('gene_type') == 'protein_coding')
        # & (F.col('enh_tSS_distance') < 20_000)
    )
    .select(
        F.col('_id.region_id').alias('region_id'),
        F.col('_id.gene_id').alias('gene_id'),
        F.col('_id.enh_id').alias('enh_id'),
        # 'dist',
        'avg_dist',
        'enh_tSS_distance'
    )
     # gene_id ENH00001.XXX -> ENH00001
    .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
    .alias("hffc6")
    .cache()
)

# hffc6_neanderthal_df_ref2 = (
#     load_distance_calculation_df([
#         {
#             '$match': {
#                 '_id.project_id': '8k_models_project_HFFC6',
#                 '_id.ensemble_id': {'$regex': '^models3D_HFFC6_Nean_models3D_HFFC6_Nean_ref2_results'},
#             }
#         }
#     ])
#     .where(
#         (F.col('enh_tSS_distance') < 20_000)
#         & (F.col('gene_type') == 'protein_coding')
#     )
#     .select(
#         F.col('_id.region_id').alias('region_id'),
#         F.col('_id.gene_id').alias('gene_id'),
#         F.col('_id.enh_id').alias('enh_id'),
#         'dist',
#         'avg_dist',
#         'enh_tSS_distance'
#     )
#      # gene_id ENH00001.XXX -> ENH00001
#     .withColumn('gene_id', F.split(F.col('gene_id'), '\.')[0])
#     .alias("hffc6_ref2")
#     .cache()
# )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<stdin>:23: SyntaxWarning: invalid escape sequence '\.'
<stdin>:77: SyntaxWarning: invalid escape sequence '\.'

In [99]:
# gm12878_neanderthal_df_ref_tested_against_ref2 = (
#     gm12878_neanderthal_df_ref
#     .join(
#         other=gm12878_neanderthal_df_ref2,
#         on=F.expr("gm12878.region_id = gm12878_ref2.region_id AND gm12878.gene_id = gm12878_ref2.gene_id AND gm12878.enh_id = gm12878_ref2.enh_id"),
#         how="inner"
#     )
#     .select(
#         gm12878_neanderthal_df_ref.region_id,
#         gm12878_neanderthal_df_ref.gene_id,
#         gm12878_neanderthal_df_ref.enh_id,
#         gm12878_neanderthal_df_ref.dist,
#         gm12878_neanderthal_df_ref.avg_dist,
#         gm12878_neanderthal_df_ref2.dist.alias('dist_ref2'),
#         gm12878_neanderthal_df_ref2.avg_dist.alias('avg_dist_ref2')
#     )
#     .withColumn('p_value', mannwhiteneyu(F.col('dist'), F.col('dist_ref2')))
# )
#
# hffc6_neanderthal_df_ref_tested_against_ref2 = (
#     hffc6_neanderthal_df_ref
#     .join(
#         other=hffc6_neanderthal_df_ref2,
#         on=F.expr("hffc6.region_id = hffc6_ref2.region_id AND hffc6.gene_id = hffc6_ref2.gene_id AND hffc6.enh_id = hffc6_ref2.enh_id"),
#         how="inner"
#     )
#     .select(
#         hffc6_neanderthal_df_ref.region_id,
#         hffc6_neanderthal_df_ref.gene_id,
#         hffc6_neanderthal_df_ref.enh_id,
#         hffc6_neanderthal_df_ref.dist,
#         hffc6_neanderthal_df_ref.avg_dist,
#         hffc6_neanderthal_df_ref.dist.alias('dist_ref2'),
#         hffc6_neanderthal_df_ref2.avg_dist.alias('avg_dist_ref2')
#     )
#     .withColumn('p_value', mannwhiteneyu(F.col('dist'), F.col('dist_ref2')))
# )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
# %%pretty
# gm12878_neanderthal_df_ref_tested_against_ref2.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

9652

In [101]:
gm12878_existing_links_df = (
    gm12878_neanderthal_df_ref
    .join(
        other=links_gm12878_df,
        on=F.expr("gm12878.gene_id = links.gene_id AND gm12878.enh_id = links.enh_id"),
        how="inner"
    )
    .select(
        gm12878_neanderthal_df_ref.region_id,
        gm12878_neanderthal_df_ref.gene_id,
        gm12878_neanderthal_df_ref.enh_id,
        gm12878_neanderthal_df_ref.avg_dist
    )
)

hffc6_existing_links_df = (
    hffc6_neanderthal_df_ref
    .join(
        other=links_hffc6_df,
        on=F.expr("hffc6.gene_id = links.gene_id AND hffc6.enh_id = links.enh_id"),
        how="inner"
    )
    .select(
        hffc6_neanderthal_df_ref.region_id,
        hffc6_neanderthal_df_ref.gene_id,
        hffc6_neanderthal_df_ref.enh_id,
        hffc6_neanderthal_df_ref.avg_dist
    )
)

gm12878_hffc6_common_links_df = (
    gm12878_existing_links_df
    .join(
        other=hffc6_existing_links_df,
        on=F.expr("gm12878.gene_id = hffc6.gene_id AND gm12878.enh_id = hffc6.enh_id"),
        how="inner"
    )
    .select(
        gm12878_existing_links_df.region_id,
        gm12878_existing_links_df.gene_id,
        gm12878_existing_links_df.enh_id,
        gm12878_existing_links_df.avg_dist.alias('gm12878_avg_dist'),
        hffc6_existing_links_df.avg_dist.alias('hffc6_avg_dist')
    )
)

gm12878_hffc6_gm12878_only_links_df = (
    gm12878_existing_links_df
    .join(
        other=hffc6_existing_links_df,
        on=F.expr("gm12878.gene_id = hffc6.gene_id AND gm12878.enh_id = hffc6.enh_id"),
        how="left_anti"
    )
    .select(
        gm12878_existing_links_df.region_id,
        gm12878_existing_links_df.gene_id,
        gm12878_existing_links_df.enh_id,
        gm12878_existing_links_df.avg_dist.alias('gm12878_avg_dist')
    )
)

gm12878_hffc6_hffc6_only_links_df = (
    hffc6_existing_links_df
    .join(
        other=gm12878_existing_links_df,
        on=F.expr("hffc6.gene_id = gm12878.gene_id AND hffc6.enh_id = gm12878.enh_id"),
        how="left_anti"
    )
    .select(
        hffc6_existing_links_df.region_id,
        hffc6_existing_links_df.gene_id,
        hffc6_existing_links_df.enh_id,
        hffc6_existing_links_df.avg_dist.alias('hffc6_avg_dist')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [92]:
gm12878_existing_links_df.count(), hffc6_existing_links_df.count(), gm12878_hffc6_common_links_df.count(), gm12878_hffc6_gm12878_only_links_df.count(), gm12878_hffc6_hffc6_only_links_df.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(13840, 20416, 3656, 10184, 16760)

In [104]:
import os

# write all to csv into /work/playground/links/experiment_2
os.makedirs("/work/playground/links/experiment_2", exist_ok=True)

gm12878_neanderthal_df_ref.toPandas().to_parquet("/work/playground/links/experiment_2/gm12878_neanderthal_df_ref.parquet", index=False)
hffc6_neanderthal_df_ref.toPandas().to_parquet("/work/playground/links/experiment_2/hffc6_neanderthal_df_ref.parquet", index=False)

gm12878_hffc6_common_links_df.toPandas().to_csv("/work/playground/links/experiment_2/gm12878_hffc6_common_links.csv", index=False)
gm12878_existing_links_df.toPandas().to_csv("/work/playground/links/experiment_2/gm12878_existing_links.csv", index=False)
hffc6_existing_links_df.toPandas().to_csv("/work/playground/links/experiment_2/hffc6_existing_links.csv", index=False)
gm12878_hffc6_gm12878_only_links_df.toPandas().to_csv("/work/playground/links/experiment_2/gm12878_hffc6_gm12878_only_links.csv", index=False)
gm12878_hffc6_hffc6_only_links_df.toPandas().to_csv("/work/playground/links/experiment_2/gm12878_hffc6_hffc6_only_links.csv", index=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…